# Using high-resolution DSMs to gain insight on forest service road sediment transport dynamics

**Lauren Wittkopf**  

**CEE 467, Geospatial Data Analysis, Winter 2023**



## Introduction

The transportation of sediment by forest service roads can have significant impacts on hydrology and sediment transport processes in forested ecosystems. Understanding these processes is critical for sustainable management of forest resources. This project is a part of Amanda Alvis' research investigating the sediment transport dynamics of forest service roads and contributes by using high-resolution digital surface models (DSMs) to parameterize a larger sediment transport model.

## Objectives

The objectives of Amanda's project are:

- To identify areas of high sediment accumulation and erosion hotspots.
- To calibrate her model outputs with field measurements.
- To investigate the impact of road design and maintenance practices on sediment transport.

The objectives of my project are: 

- To make a data analysis product that takes an input of a GeoTiff DSM in an arbitrary coordinate system and produces figures that aid in sediment transport analysis 

## Methods

I have used the following methods to achieve our objectives:

### DSM Creation and Reprojection

I have used Pix4D to create high-resolution DSMs of the study area that are in an arbitrary coordinate system. This allows me to export a GeoTiff after 3 steps of processing. The arbitrary CRS must be reprojected using `gdalwarp` into EPSG:32610. This process will be repeated for multiple flight dates to generate a time series of DSMs.

### Adding Overviews and Clipping

I added gaussian overviews to these reprojected DEMs using `gdaladdo` and then opened them using rioxarray to produce xArray DataSets which could then be clipped to a QGIS-drawn polygon (in EPSG:32610) using `xds.rio.clip(mask_gdf.geometry)` where xds is the xArray DataSet and mask_gdf is the shapefile loaded using `gpd.read_file()`. I also extracted the extent for plotting all the data from the mask_gdf using the `mask_gdf.bounds` attribute.

